In [1]:
from IPython.display import display, Image
import cv2 as cv
from PIL import Image

#august
import os
import pandas as pd
import numpy as np


#!pip install tensorflow
import tensorflow
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder

In [2]:
#display(Image(filename='brad.jpg'))
#display(Image(filename='kim.jpg'))
face_cascade = cv.CascadeClassifier("haarcascade_frontalface_default.xml")
eye_cascade = cv.CascadeClassifier("haarcascade_eye.xml")

In [18]:
def coords_for_crop(img_address):

    img = cv.imread(img_address)
    im = Image.open(img_address) 

    gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray)

    xUpper, yUpper, xLower, yLower = 0,0,0,0
    
    

    for (x, y, w, h) in faces:
        #print(w,h,type(w),type(h))
        if (w>200 and h>200):
            return x,y,x+w,y+h
        
        
        #cv.rectangle(img, (x, y), (x+w, y+h), (0, 0, 255), 2)
        #roi_gray = gray[y : y + h, x : x + h]
        #roi_color = img[y : y + h, x : x + h]
        # Otsi silmi vaid leitud nägude aladelt (kollased ristkülikud)
        #eyes = eye_cascade.detectMultiScale(roi_gray)
        #for (ex, ey, ew, eh) in eyes:
            #cv.rectangle(roi_color, (ex, ey), (ex+ew, ey+eh), (0, 255, 255), 2)


    # Otsi silmi kogu fotolt (rohelised ristkülikud)
    #eyes = eye_cascade.detectMultiScale(gray)
    #for (ex, ey, ew, eh) in eyes:
        #cv.rectangle(img, (ex, ey), (ex+ew, ey+eh), (0, 255, 0), 2)

    #im1 = im.crop((xUpper, yUpper, xLower, yLower))
    #im1.show()
    #cv.waitKey(0)
    #cv.destroyAllWindows()

In [4]:
# Pildi salvestamiseks
#cv.imwrite('jott_koopia.png', img)

In [5]:
import os
import pandas as pd
from PIL import Image
from numpy import asarray

# Defineerime CSV faili tee ja piltide peamise kausta tee
csv_file_path = './ratings.csv'
images_folder_path = './people'

# Laadime CSV faili
df = pd.read_csv(csv_file_path, header=None)

# Loome tühja DataFrame, kuhu salvestame ümberstruktureeritud andmed
columns = ['usaldusväärsus', 'ilu', 'intelligents', "pilt"]
andmestik_df = pd.DataFrame(columns=columns)

# Itereerime läbi CSV andmete ja lisame need ümberstruktureeritult uude DataFrame'i
for index, row in df.iterrows():
    # Määrame kausta nime vastavalt rea indeksile
    folder_name = f'id{index + 1}'
    folder_path = os.path.join(images_folder_path, folder_name)
    
    # Lisame andmed vastavalt kaustas olevatele piltidele
    for i in range(0, len(row)-3, 3):
        image_name = f'Allalaaditud fail ({i//3 + 1}).jpeg'  # Oletame, et pildid on nimedega img1.jpg, img2.jpg, img3.jpg
        image_path = os.path.join(folder_path, image_name)
        
        # load the image and convert into
        # numpy array
        img = Image.open(image_path)
        
        #crop image so it will only contain the face of the person
        img = img.crop(coords_for_crop(image_path))
        
        #resize image to 600x600 
        img = img.resize((600,600))
        
        ##
        #img.show()
        ##
        
        # asarray() class is used to convert
        # PIL images into NumPy arrays
        image_array = asarray(img)
        
        
        image_data = pd.DataFrame({
            'usaldusväärsus': [row[i]],
            'ilu': [row[i+1]],
            'intelligents': [row[i+2]],
            'pilt': [image_array]  
        })
        andmestik_df = pd.concat([andmestik_df,image_data], ignore_index=True)


# Kui soovite tulemused salvestada uude CSV faili
andmestik_df.to_csv('reshaped_file.csv', index=True)
# Vaatame ümberstruktureeritud andmeid
print(andmestik_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 210 entries, 0 to 209
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   usaldusväärsus  180 non-null    float64
 1   ilu             180 non-null    float64
 2   intelligents    180 non-null    float64
 3   pilt            210 non-null    object 
dtypes: float64(3), object(1)
memory usage: 6.7+ KB
None


In [6]:
# Load the data
#df = pd.read_csv('reshaped_file.csv')  # assuming your data is in a CSV file
df = andmestik_df

# Drop rows with missing ratings
df = df.dropna(subset=['usaldusväärsus', 'ilu', 'intelligents'])

# Extract the images and labels
images = np.stack(df['pilt'].values)
ratings = df[['usaldusväärsus', 'ilu', 'intelligents']].values

# Normalize image data
images = images / 255.0

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(images, ratings, test_size=0.2, random_state=42)

# Define the CNN model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(600, 600, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(3)  # Output layer for the 3 ratings
])

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])

# Train the model
history = model.fit(X_train, y_train, epochs=25, batch_size=16, validation_split=0.2)

# Evaluate the model
loss, mae = model.evaluate(X_test, y_test)
print(f'Model evaluation - Loss: {loss}, MAE: {mae}')

# Save the model
model.save('face_ratings_model.h5')

C:\Users\kost\AppData\Local\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/25
8/8 ━━━━━━━━━━━━━━━━━━━━ 60s 6s/step - loss: 6487.5068 - mae: 43.1250 - val_loss: 22.6165 - val_mae: 4.4275
Epoch 2/25
8/8 ━━━━━━━━━━━━━━━━━━━━ 43s 5s/step - loss: 12.2090 - mae: 2.9062 - val_loss: 5.7639 - val_mae: 2.0483
Epoch 3/25
8/8 ━━━━━━━━━━━━━━━━━━━━ 61s 7s/step - loss: 7.3075 - mae: 2.2267 - val_loss: 5.1407 - val_mae: 1.9980
Epoch 4/25
8/8 ━━━━━━━━━━━━━━━━━━━━ 44s 5s/step - loss: 5.3143 - mae: 1.8987 - val_loss: 4.1208 - val_mae: 1.7628
Epoch 5/25
8/8 ━━━━━━━━━━━━━━━━━━━━ 45s 5s/step - loss: 5.0762 - mae: 1.8164 - val_loss: 3.4902 - val_mae: 1.4383
Epoch 6/25
8/8 ━━━━━━━━━━━━━━━━━━━━ 49s 6s/step - loss: 5.9094 - mae: 2.0152 - val_loss: 3.7966 - val_mae: 1.6700
Epoch 7/25
8/8 ━━━━━━━━━━━━━━━━━━━━ 48s 6s/step - loss: 4.9457 - mae: 1.8101 - val_loss: 3.9803 - val_mae: 1.7324
Epoch 8/25
8/8 ━━━━━━━━━━━━━━━━━━━━ 47s 6s/step - loss: 4.8861 - mae: 1.8457 - val_loss: 5.8209 - val_mae: 2.1135
Epoch 9/25
8/8 ━━━━━━━━━━━━━━━━━━━━ 52s 6s/step - loss: 5.6291 - mae: 1.9959 - val

In [35]:
# Load the trained model
model = load_model('face_ratings_model.h5')

# Function to preprocess the image
def preprocess_image(image_path):
    image = Image.open(image_path)
    
    #image = image.crop(coords_for_crop(image_path))
    image = image.resize((600, 600))  # Ensure the image is 600x600 pixels
    image.show()
    image = np.array(image) / 255.0  # Normalize the image
    image = np.expand_dims(image, axis=0)  # Add batch dimension
    return image


def display_results(image_path):
    # Path to the random image
    random_image_path = image_path  # Replace with the actual path

    # Preprocess the image
    preprocessed_image = preprocess_image(random_image_path)

    # Predict the ratings
    predicted_ratings = model.predict(preprocessed_image)

    # Print the predicted ratings
    usaldusvaarus, ilu, intelligents = predicted_ratings[0]
    print(f"Predicted Reliability (usaldusväärsus): {usaldusvaarus}")
    print(f"Predicted Attractiveness (ilu): {ilu}")
    print(f"Predicted Intelligence (intelligents): {intelligents}")

In [31]:
print("tyler ratings:")
display_results('people/tyler.jpg')

print("kim ratings:")
display_results('people/kim.jpg')

print("brad ratings:")
display_results('people/brad.jpg')

print("homeless ratings:")
display_results('people/homeless.png')


tyler ratings:
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 258ms/step
Predicted Reliability (usaldusväärsus): 4.001458644866943
Predicted Attractiveness (ilu): 4.326530456542969
Predicted Intelligence (intelligents): 4.633413314819336
kim ratings:
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 130ms/step
Predicted Reliability (usaldusväärsus): 4.8442301750183105
Predicted Attractiveness (ilu): 7.144835948944092
Predicted Intelligence (intelligents): 7.220454692840576
brad ratings:
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 129ms/step
Predicted Reliability (usaldusväärsus): 7.994103908538818
Predicted Attractiveness (ilu): 8.526636123657227
Predicted Intelligence (intelligents): 7.176937580108643
homeless ratings:
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 116ms/step
Predicted Reliability (usaldusväärsus): 3.338634490966797
Predicted Attractiveness (ilu): 4.160274505615234
Predicted Intelligence (intelligents): 4.202895164489746


In [36]:
print("just some dude ratings:")
display_results('people/girl.png')

print("man ratings:")
display_results('people/old_man.png')

print("airbrushed face ratings:")
display_results('people/airbrushed.jpg')

just some dude ratings:
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 339ms/step
Predicted Reliability (usaldusväärsus): 6.090631484985352
Predicted Attractiveness (ilu): 6.4580159187316895
Predicted Intelligence (intelligents): 6.231784820556641
man ratings:
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 122ms/step
Predicted Reliability (usaldusväärsus): 4.635649681091309
Predicted Attractiveness (ilu): 5.2997660636901855
Predicted Intelligence (intelligents): 5.339832305908203
airbrushed face ratings:
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 124ms/step
Predicted Reliability (usaldusväärsus): 5.652440071105957
Predicted Attractiveness (ilu): 5.758391857147217
Predicted Intelligence (intelligents): 5.533234119415283
